In [1]:
using TensorNetworks, DoubleFloats, LinearAlgebra, TensorOperations, Plots
plotlyjs()
theme(:dark)

ErrorException: Failed to precompile TensorNetworks [bf05fd6d-b814-440f-8e78-cc5435f5625f] to C:\Users\svens\.julia\compiled\v1.7\TensorNetworks\jl_8614.tmp.

In [2]:
T = Complex{Double64}
Dmax = 50;
tol = 1e-22;
d=2
truncation = TruncationArgs(Dmax,tol,true)
truncationnonormalize = TruncationArgs(Dmax,tol,false)
# truncation = TruncationArgs(Dmaxhigh,tol,false)

UndefVarError: UndefVarError: Double64 not defined

In [34]:
# Add in the transitional layer of https://arxiv.org/pdf/1602.01166.pdf
transition_gate = Gate(reshape(II + 1im*kron(sx,sx),2,2,2,2)/sqrt(2))
transition_layers = fill(fill(TensorNetworks.auxillerate(transition_gate,transition_gate),2),2);

In [21]:
w, u = TensorNetworks.constr_wu_dim2(T)
@tensor U[:] := u[-1, -3, -5, -7] * u[-2, -4, -6, -8]
d = size(u, 1)
U = Gate(reshape(U, d^2, d^2, d^2, d^2))
@tensor W[:] := w[-1, -3, -5] * w[-2, -4, -6]
W = reshape(W, d^2, d^2, d^2);

In [26]:
function coarse_grain!(Λs,eigs, mps,cg, U, W)
    # println("norm: ", norm(mps))
    if length(mps) == 1
        mps = TensorNetworks.double(mps)
    end
    println("A: ", sqrt(transfer_spectrum(mps)[1][1]))
    mps = TensorNetworks.apply_layers_nonunitary(mps, [[U]])
    println(sqrt(transfer_spectrum(mps)[1][1]))
    #println(norm.(mps.Λ))
    # println("dnorm: ", norm(mps))
    A = data(mps.Γ[1])
    B = data(GenericSite(mps[2],:right))
    # lA = mps.Λ[1]
    lB = mps.Λ[2]
    # B = B*lA
    @tensor C[:] := B[-1, ul, c] * W[-2, ul, ur] * A[c, ur, -3]
    newmps = UMPS([GenericSite(C,false)], [lB], mps)
    push!(eigs, sqrt(transfer_spectrum(newmps)[1][1]))
    if cg
        newmps.truncation = truncation
    end
    newcanmps = canonicalize(newmps)
    
    push!(Λs, real.(vec(newcanmps.Λ[1])))
    newcanmps.truncation = truncationnonormalize
    return newcanmps
end
coarse_grain_uw!(Λs,eigs, mps,cg) = coarse_grain!(Λs,eigs, mps,cg,U,W)

coarse_grain_uw! (generic function with 1 method)

In [27]:
A0 = A = B = reshape(
    permutedims(
        reshape(kron(Matrix{T}(I, d, d), Matrix{T}(I, d, d)), d, d, d, d),
        [1, 3, 2, 4]),
    d,d^2,d)
lA0 = lA = lB = ones(T,d) / sqrt(d)  #(x->x/norm(x))(ComplexF64.(rand(d))) 

mps = UMPS([A0], [lA0], truncation=truncationnonormalize)

MPS: UMPS{ComplexF64}
Sites: OrthogonalLinkSite{ComplexF64}
Length: 1
Truncation: TruncationArgs(300, 1.0e-14, false)

In [35]:
t_mps = TensorNetworks.apply_layers(canonicalize(TensorNetworks.double(copy(mps))), transition_layers)

MPS: UMPS{ComplexF64}
Sites: OrthogonalLinkSite{ComplexF64}
Length: 2
Truncation: TruncationArgs(300, 1.0e-14, false)

In [512]:
mps2 = canonicalize(copy(mps))
# mps2.Λ = 10*mps2.Λ 
mps21 = (coarse_grain_uw!([],[],mps2,false))
mps21n = (coarse_grain_uw!([],[],mps2,true))
s1 = transfer_spectrum(mps21)[1][1] / transfer_spectrum(mps21n)[1][1]
mps22 = (coarse_grain_uw!([],[],mps21,false))
mps2n2 = (coarse_grain_uw!([],[],mps21n,false))
mps22n = (coarse_grain_uw!([],[],mps21n,true))
s2 = transfer_spectrum(mps22)[1][1] / transfer_spectrum(mps22n)[1][1]
s12 = transfer_spectrum(mps2n2)[1][1] / transfer_spectrum(mps22n)[1][1]

println(s1)
println(s12)
println(s12*s1^2)
println(s2)

0.9999999999999999
1.0
0.39895992071442604 + 0.0im
0.6706260725650062 + 4.235164736271483e-22im
0.10674289364100836 + 6.741070135655292e-23im
0.10674289364100796 - 1.6263032587282494e-19im


In [517]:
eigsnormalize[1]*eigsnorma

5-element Vector{Any}:
 0.6316327419588271 + 3.461008006649979e-20im
 0.8189176225756825 + 2.5680947987095093e-21im
 0.7740451672333832 - 1.7508703276978884e-19im
 0.7299414724223495 + 3.7133188531113393e-20im
 0.7141465010384518 + 1.1119474602146798e-22im

In [526]:
absolute_scaling(eigsnormalize) ./ eigsnonormalize

10-element Vector{ComplexF64}:
                    1.0 + 0.0im
     0.3989599207144263 + 4.3721719543635625e-20im
    0.10674289364100878 + 2.406523125954434e-20im
  0.0068266957899194005 - 1.0199177338532011e-23im
   2.483116976973576e-5 + 2.4521995093268127e-24im
 3.1446258762146125e-10 + 6.220736380670996e-29im
  4.973324266152988e-20 + 1.2640712693743192e-38im
 1.2387636109799192e-39 + 6.386177632051497e-58im
   7.67624942554941e-79 + 9.578394924290677e-97im
 2.946616406648534e-157 + 7.353553556854074e-175im

In [39]:
function absolute_scaling(eigs)
    temp = one(eigs[1])
    next(prev,e) = prev^2*e
    return accumulate(next,eigs,init=1)
end

absolute_scaling (generic function with 1 method)

In [40]:
Λs = []
states = []
eigs = []
mpstemp = canonicalize(copy(t_mps))
for k in 1:30
    mpstemp=coarse_grain_uw!(Λs,eigs, mpstemp,true)
    push!(states,mpstemp)
end

A: 0.9999999999999996 - 6.938893903907231e-18im
0.9999999999999999 + 8.673617379884043e-18im
A: 0.9999999999999999 - 3.903127820947817e-18im
1.0000000000000007 - 3.0357660829594105e-18im
A: 1.0000000000000009 - 8.02987233997076e-19im
0.9999999999999969 + 1.0933366299265722e-18im
A: 1.0000000000000056 + 8.046812998915809e-20im
1.0000000000000049 + 1.4393691456178507e-19im
A: 1.0000000000000038 + 3.557538378468048e-20im
1.0000000000000044 - 6.0253437048705376e-18im
A: 0.999999999999997 - 8.122913615270756e-22im
0.9999999999999984 - 1.753879739675087e-19im
A: 1.0000000000000044 - 2.2499312661442254e-22im
1.0000000000000047 + 1.7865335213406214e-18im
A: 1.0000000000000016 - 2.9985297205047206e-23im
0.9999999999999922 + 2.2332248647486414e-18im
A: 1.0000000000000027 - 2.0679515313825637e-25im
1.0000000000000069 - 1.8228149024912416e-18im
A: 0.999999999999998 - 2.0557084101335432e-25im
0.999999999999997 - 2.6156377928200754e-18im
A: 0.9999999999999967 - 6.032198461171636e-25im
0.999999999999

In [567]:
eigsnonormalize=[]
mpstemp = canonicalize(copy(mps))
for k in 1:5
    mpstemp=coarse_grain_uw!([],eigsnonormalize, mpstemp,false)
    push!(states,mpstemp)
end

In [41]:
length.(Λs)

30-element Vector{Int64}:
  8
 29
 36
 32
 25
 22
 20
 16
 16
 16
 13
 13
 11
  ⋮
  6
  6
  7
  7
  7
  7
  7
  7
  7
  7
  7
  7

In [43]:
ind = 2:length(Λs)-2
plot(ind, getindex.(Λs[ind],[1 2 3 4 5 6]), yaxis=:log10, size= (1200,600))

In [644]:
ind = 2:length(Λs)-2
plot(ind, getindex.(Λs[ind],[1 2 3 4 5 6 7]), yaxis=:log10, size= (1200,600))

In [51]:
maximum(abs.(states[end].Γ[1]))

6.367040428530028e9

In [668]:
v = zeros(49)
v[1] = 1
vals,vecs = eigen(Matrix(transfer_matrix(states[end])));

In [52]:
plot([maximum(abs.(GenericSite(state[1],:right)))  for state in states])

In [685]:
maximum(abs.(states[4].Γ[1]))

6.368021300554675e9

In [55]:
heatmap(real.(Matrix(transfer_matrix(states[end-1]))))

In [658]:
heatmap(real.(Matrix(transfer_matrix(states[end]))))

In [649]:
abs.(states[end].Γ[1] ./ states[end-1].Γ[1])[1,:,1]

4-element Vector{Float64}:
 0.9999999999999988
 1.8147241370464946
 1.8147241354198718
 0.9999999999999971

In [57]:
real.(eigs)

30-element Vector{Float64}:
 0.631632741958827
 0.8189176225756799
 0.7740451672333802
 0.7299414724223539
 0.7141465010384539
 0.7091766120031356
 0.7076971441450541
 0.7072714044969315
 0.7071519176663601
 0.7071189938335242
 0.7071100510089654
 0.7071076491755349
 0.7071070099717804
 ⋮
 0.7071067812560453
 0.7071067812043621
 0.7071067811910976
 0.7071067811877133
 0.7071067811868549
 0.7071067811866306
 0.7071067811865689
 0.707106781186553
 0.7071067811865499
 0.7071067811865506
 0.7071067811865486
 0.7071067811865495

In [56]:
absolute_scaling(real.(eigs))

30-element Vector{Float64}:
 0.631632741958827
 0.32671530977443963
 0.08262382095932899
 0.004983088376673143
 1.773309300774807e-5
 2.230095124911145e-10
 3.519607380063389e-20
 8.761420789772798e-40
 5.428274501760362e-79
 2.0836084286809942e-157
 3.0698646055e-314
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [616]:
absolute_scaling(real.(eigs))

20-element Vector{Float64}:
 0.6387493740101703
 0.18799022239204238
 0.015407027033964139
 0.00013741696313981664
 1.2564362024986311e-8
 1.1162834171636848e-16
 9.04999026051829e-33
 6.02276124264059e-65
 2.6834122625072957e-129
 5.342491091916393e-258
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [608]:
collect(enumerate(2:5))

4-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (2, 3)
 (3, 4)
 (4, 5)

In [612]:
[ log2(x^1/(2n)) for (n,x) in enumerate(absolute_scaling(real.(eigs)))]

20-element Vector{Float64}:
    -1.662842136739713
    -3.61389403430131
    -6.182260910964984
   -10.648744123753717
   -19.105124376135336
   -35.647138199847944
   -68.43050231776816
  -133.7459589530478
  -264.1617508194699
  -524.8055548139519
 -1045.926678385224
   -Inf
   -Inf
   -Inf
   -Inf
   -Inf
   -Inf
   -Inf
   -Inf
   -Inf

In [579]:
log.(absolute_scaling(abs.(eigs[1:11])))

11-element Vector{Float64}:
   -0.4594471582374618
   -1.1186660995984685
   -2.4934572506932824
   -5.301705424204338
  -10.940078002571246
  -22.223806688467327
  -44.79335241624655
  -89.93304563735147
 -180.2126010343982
 -360.7717583879346
 -721.8900857419493

In [603]:
-1.51(2 .^((1:11) .- 2.1))

11-element Vector{Float64}:
   -0.7044399086102896
   -1.4088798172205792
   -2.8177596344411584
   -5.635519268882317
  -11.271038537764634
  -22.542077075529267
  -45.08415415105854
  -90.16830830211708
 -180.33661660423417
 -360.67323320846833
 -721.3464664169367

In [617]:
diff(log2.(abs.(log.(absolute_scaling(abs.(eigs[1:11]))))))

10-element Vector{Float64}:
  1.8986737746740663
  1.3200341523334105
  1.091526301335195
  1.0326765247230307
  1.0136761891840802
  1.0062673571590262
  1.0030021873679331
  1.001469617136241
  1.0007271095844263
 Inf

In [58]:
plot((log10.(absolute_scaling(abs.(eigs[1:5])))))

In [163]:
plot(cumprod(scalings),xaxis=:log10)

┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\svens\.julia\packages\PlotUtils\VgXdq\src\ticks.jl:294
┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\svens\.julia\packages\PlotUtils\VgXdq\src\ticks.jl:294


┌ Warning: No strict ticks found
└ @ PlotUtils C:\Users\svens\.julia\packages\PlotUtils\VgXdq\src\ticks.jl:294
